Import libraries and data loading

In [1]:
!pip install nltk
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd
import numpy as np
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\c23000909\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\c23000909\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

def load_data_from_folders(folder_paths):
    data = []
    labels = []
    for folder_path in folder_paths:
        category = os.path.basename(folder_path)  # Extract category name from folder path
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='latin-1') as file:
                text = file.read()
                data.append(text)
                labels.append(category)
    return data, labels

In [4]:
folder_paths = ["bbc/tech", "bbc/sport", "bbc/politics", "bbc/entertainment", "bbc/business"]
data, labels = load_data_from_folders(folder_paths)
df = pd.DataFrame({'text': data, 'category': labels})

In [5]:
print(df)

                                                   text  category
0     Ink helps drive democracy in Asia\n\nThe Kyrgy...      tech
1     China net cafe culture crackdown\n\nChinese au...      tech
2     Microsoft seeking spyware trojan\n\nMicrosoft ...      tech
3     Digital guru floats sub-$100 PC\n\nNicholas Ne...      tech
4     Technology gets the creative bug\n\nThe hi-tec...      tech
...                                                 ...       ...
2220  Trial begins of Spain's top banker\n\nThe tria...  business
2221  UK economy ends year with spurt\n\nThe UK econ...  business
2222  HealthSouth ex-boss goes on trial\n\nThe forme...  business
2223  Euro firms miss out on optimism\n\nMore than 9...  business
2224  Lacroix label bought by US firm\n\nLuxury good...  business

[2225 rows x 2 columns]


Data loaded succesfully

Preprocessing 

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Step 2: Data Preprocessing
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing Punctuation
    tokens = [token for token in tokens if token not in string.punctuation]
    
    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text


df['text'] = df['text'].apply(preprocess_text)
print(df.head())


                                                text category
0  ink help drive democraci asia kyrgyz republ sm...     tech
1  china net cafe cultur crackdown chines author ...     tech
2  microsoft seek spywar trojan microsoft investi...     tech
3  digit guru float sub- 100 pc nichola negropont...     tech
4  technolog get creativ bug hi-tech art world ti...     tech


In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np

# Step 3: Feature Engineering

# 1. Word Frequency Feature
count_vectorizer = CountVectorizer()
word_freq_features = count_vectorizer.fit_transform(df['text'])

# 2. TF-IDF Feature
tfidf_vectorizer = TfidfVectorizer()
tfidf_features = tfidf_vectorizer.fit_transform(df['text'])

# 3. N-grams Feature
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))
ngram_features = ngram_vectorizer.fit_transform(df['text'])

#Compute Cosine Similarity between documents
cos_sim_features = cosine_similarity(tfidf_features)

In [14]:
from scipy.sparse import hstack

# 1. Combine Features
combined_features = hstack([word_freq_features, tfidf_features, ngram_features, cos_sim_features])
print("Combined Features Matrix Shape:", combined_features.shape)

Combined Features Matrix Shape: (2225, 384108)


In [15]:
from sklearn.feature_selection import SelectKBest, chi2

# Here, we'll select the top 3000 features
y = df['category']  
k_best_selector = SelectKBest(score_func=chi2, k=3000)
selected_features = k_best_selector.fit_transform(combined_features, y)


In [16]:
X = selected_features  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Model Selection
classifier = SVC(kernel='linear')  
# 3. Model Training
classifier.fit(X_train, y_train)



SVC(kernel='linear')

In [17]:

from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report


k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Perform cross-validation and get accuracy scores for each fold
cv_scores = cross_val_score(classifier, X_train, y_train, cv=kfold)
print("Cross-Validation Scores:", cv_scores)

# Calculate and print the mean accuracy and standard deviation
print("Mean Accuracy:", np.mean(cv_scores))
print("Standard Deviation of Accuracy:", np.std(cv_scores))

# Generate predictions on the test data using the trained classifier
y_pred = classifier.predict(X_test)

# Generate a classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print("\nClassification Report:")
print(report)


Cross-Validation Scores: [0.95505618 0.96910112 0.97191011 0.96910112 0.96629213]
Mean Accuracy: 0.9662921348314606
Standard Deviation of Accuracy: 0.005892184540281754

Classification Report:
               precision    recall  f1-score   support

     business       0.92      0.94      0.93       102
entertainment       0.99      0.97      0.98        76
     politics       0.92      0.91      0.92        78
        sport       0.97      1.00      0.99       104
         tech       1.00      0.96      0.98        85

     accuracy                           0.96       445
    macro avg       0.96      0.96      0.96       445
 weighted avg       0.96      0.96      0.96       445

